# Lecture 17 (5/4/2022)

**Announcements**


*Last time we covered:*
- Prediction: regression + overfitting

**Today's agenda:**
- Classification! 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Classification

## Overview

- What problem it solves
- Contrast with regression
- Context: widespread, huge part of most machine learning curricula
- Goals: data-based (doesn't require specification by a human), accurate incl. w/ new data, fair (!!)


## Structure

Ed lec 7 "problem statement"



## Examples
- *Binary* classification
- *Multiclass* classification
- Other: *one-class*, *multi-label*




- For each one: how do people solve this? how might you build an algorithm to solve it?


Binary prediction
- Toy: Titanic, spam emails, CAPTCHA; real world: risk assessment tools, loan defaults, voter behavior, ...

Generalizing to multiclass
- Toy: MNIST; real world: face recognition

### Binary classification



![recidivism_classifier](img/recidivism.png)
([source](https://www.technologyreview.com/2019/01/21/137783/algorithms-criminal-justice-ai/), see also [here](https://www.nytimes.com/2020/02/06/technology/predictive-algorithms-crime.html))

### Multiclass classification



![fb](img/fb.png)

([source](https://www.nytimes.com/2021/11/02/technology/facebook-facial-recognition.html))

### Other types of classification (less common)

We won't discuss these in detail but it's helpful to know they exist! 

**Multi-label classification**: Decide which among a set of multiple labels should be applied to a particular instance 
- *Does this image contain a cat? A person? A car? All the above?*

**One-class classification**: Try to optimally identify an instance of a single class 
- *Is this datapoint an outlier? Does this medical scan contain a tumor?*


## Intuitive solution: $k$-nearest neighbors (k-NN)

- How it works
- `sklearn` implementation
- Try it out!
- How well does it perform? *...next time!*

### How it works

![mean_girls_cafeteria](img/mean_girls.jpeg)

*Birds of a feather flock together*: the k-NN classifier guesses an item's label based on whatever training data the item is *closest to*.


**Pseudo-code**

Steps for classifying a *new* example $x$ with *unknown* label $y$ (more [here](https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761)):
1. Cycle through all *known* data points $x_i$ with *known label* $y_i$.
2. For each known data point $x_i$, calculate and store the *distance* from $x_i$ to our unknown data point $x$ (Note: this algorithm relies on a distance function $D(x_1, x_2)$)
3. After cycling through all known data points, select the $k$ data points with the *lowest* distance to our unknown data point $x$. These are the "nearest neighbors." We choose $k$ ahead of time!
4. Select an estimated label $\hat{y}$ for $x$ that corresponds to the *most common label* $y$ among the nearest data points from step 3.


![knn_wikipedia](img/knn.png)

In the image above, we're trying to guess a classification (*red triangle* or *blue square* for the green circle. The black outlines show our *nearest neighbors* for $k = 3$ and $k = 5$ ([source](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm))

Let's see what this looks like in practice!

### Implementing k-NN classification

In [ ]:
# The iris dataset is a popular dataset for illustrating classification algorithms
iris = sns.load_dataset('iris')
iris


In [ ]:
# Let's look at whether petal length and petal width allow for straightforward k-NN classification

sns.scatterplot(data = iris, x = "petal_length", y = "petal_width", hue = "species", alpha = 0.5)
plt.show()

In [ ]:
# Let's set aside some test data that we'll classify

from sklearn.model_selection import train_test_split

x_vals = iris.loc[:, ('petal_length', 'petal_width')]
y_vals = iris.loc[:, 'species']

xtrain, xtest, ytrain, ytest = train_test_split(x_vals, 
                                               y_vals, 
                                               test_size = 0.1, # we'll use a small test set here
                                               random_state = 0
                                               )

xtrain = xtrain.reset_index(drop = True)
xtest = xtest.reset_index(drop = True)
ytrain = ytrain.reset_index(drop = True)
ytest = ytest.reset_index(drop = True)



In [ ]:
# For illustration we'll pick a single element from the test set to classify
test_x = xtest.iloc[8, :]

# Where is it alongside the training data?
sns.scatterplot(data = xtrain, x = "petal_length", y = "petal_width", hue = ytrain, alpha = 0.5)
plt.text(s = "X", x = test_x[0], y = test_x[1], horizontalalignment = 'left')
plt.show()

*Can we classify the element labeled with the X above?*

In [ ]:
# For distance, we'll use Euclidean distance
xtrain['distance'] = np.sqrt((xtrain['petal_length'] - test_x[0])**2 + (xtrain['petal_width'] - test_x[1])**2)
xtrain

![knn_distance](img/knn_dist.png)

In [ ]:
# Set our k value
k = 3

# What are the values for the k smallest distances?
xtrain.nsmallest(n = k, columns = 'distance', keep = "all")

In [ ]:
# What are the classifications of these nearest neighbor items?
ytrain.iloc[list(xtrain.nsmallest(n = k, columns = 'distance').index)]

Our $k$-nearest neighbors above classified our test item as `versicolor`.

*Was this classification accurate?*

In [ ]:
test_y = ytest.iloc[8]
test_y

*What if we change $k$ to 5? 10? 25?*

The process above should give you an intuition for what the k-NN classifier needs to do to classify novel test data. Fortunately, the `sklearn` `KNeighborsClassifier` will do most of this work for us!

### k-NN classification with `sklearn`


In [ ]:
# Import the KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Initialize the classifier (use the same `k` as above)
knn = KNeighborsClassifier(n_neighbors = k)

# Re-generate the train/test data we used above
xtrain, xtest, ytrain, ytest = train_test_split(x_vals, 
                                                y_vals, 
                                                test_size = 0.1, # we'll use a small test set here
                                                random_state = 0
                                               ) 

# Use the model's `fit` function to train the classifier
# Question: what is this "fit" function doing for kNN?
knn.fit(X = xtrain, y = ytrain)

In [ ]:
# Now, let's predict our test elements! 
# (we can do them all at once instead of one at a time as we did above...)

# reminder: this is what our test data looks like
xtest
ytest

# use the model's `predict` function to make predictions about xtest
preds = knn.predict(X = xtest)
preds

*How did we do?*

In [ ]:
knn_eval = pd.DataFrame({
    "petal_length": xtest['petal_length'],
    "petal_width": xtest['petal_width'],
    "species": ytest,
    "knn prediction": preds
})

knn_eval

### Your turn! 

Use the `sklearn` `KNeighborsClassifier` to classify a set of held out test data.

But this time, use `sepal_length` and `sepal_width` as the **features** instead of *petal length* and *petal width* as we did above. This makes things a little more interesting...

In [ ]:
sns.scatterplot(data = iris, x = "sepal_length", y = "sepal_width", hue = "species", alpha = 0.5)
plt.show()

In [ ]:
### YOUR CODE HERE





## Closing thoughts

In the above, we showed how the $k$-nearest neighbors classifier generates predictions for classifying novel data.

Next week, we'll look at other solutions to this problem and how to compare them.

But first, we need to cover an important question for *any* classifier: how do we evaluate the output to determine whether it did a good job (or, e.g., choose the best $k$ value)?

This is what we'll discuss on Friday!

## Classification by hand

Pick a threshold and evaluate TP/FP etc. 
Or pick a loss function (FP very bad!) and find optimal threshold?
Vul lecture 8

## Classification models
- K nearest neighbors
- Logistic regression
- Decision trees, SVMs, discriminant analysis, neural networks, random forests, gradient boosted trees 

vul lecture 8, Garrett lecture 16, 17

## Evaluating classification
- confusion matrix (Garrett binary classification ppt)
- FP, TP, FA, FN, ... (Garrett binary classification ppt)
- Metrics: accuracy, precision, F1, ... (Garrett binary classification ppt)
- Thresholds and ROC curves: evaluating ROC curves (AUC) (Garrett lec 15)

Vul lecture 7, 7-extras; 

## Classification: understanding the problem
- Common examples: labels (MNIST, 538 margaritas? social science example?), binary (titanic, but also loan defaults, voter behavior!)
- Solutions: use rules (decision tree), learn a function (logistic, neural networks), draw a line (svm, discriminant), use clustering info (knn)
